In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium

# minus 폰트 깨짐 방지
plt.rc('axes', unicode_minus = False)

#한글 폰트 설정
plt.rc('font', family='NanumGothic')

# retina 설정
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('retina')

# csv 파일 불러오기
df = pd.read_csv("data file/소상공인시장진흥공단_상가업소정보_의료기관_201909.csv", low_memory = False)

In [2]:
# 종합병원데이터 전처리
df_seoul_hospital = df[(df["상권업종소분류명"] == "종합병원") & (df["시도명"] == "서울특별시")].copy()

drop_row = df_seoul_hospital[
df_seoul_hospital["상호명"].str.contains("꽃배달|의료기|장례식장|상담소|어린이집")].index

drop_row = drop_row.tolist()
drop_row

drop_row2 = df_seoul_hospital[df_seoul_hospital["상호명"].str.endswith("의원")].index
drop_row2.tolist()
drop_row2

print(df_seoul_hospital.shape)
df_seoul_hospital = df_seoul_hospital.drop(drop_row, axis=0)
df_seoul_hospital = df_seoul_hospital.drop(drop_row2, axis=0)
print(df_seoul_hospital.shape)

(91, 39)
(58, 39)


scatter plot은 수치형 데이터가 어디 좌표에 위치하는지 출력할 때 주로 이용됩니다. 상관계수, 회귀선을 출력하는데 사용하는데 지리 데이터등 다양하게 이용하능 하지만 정확한 위치는 알기 어렵습니다. 그래서 Folium map을 사용하여 직관적으로 정확한 위치를 알아보도록 하겠습니다.

In [3]:
df_seoul_hospital["위도"].mean()
df_seoul_hospital["경도"].mean()

126.9963589356625

In [4]:
map = folium.Map(location=[df_seoul_hospital["위도"].mean(),df_seoul_hospital["경도"].mean()], zoom_start=1)

folium.Map만 찍으면 세계지도가 나오지만 location과 zoom_start를 이용하여 위치를 설정할 수 있습니다.

In [8]:
for n in df_seoul_hospital.index:
    name = df_seoul_hospital.loc[n, "상호명"]
    address = df_seoul_hospital.loc[n, "도로명주소"]
    popup = f"{name}-{address}"
    location = [df_seoul_hospital.loc[n, "위도"], df_seoul_hospital.loc[n, "경도"]]
    folium.Marker(
        location = location,
        popup = popup,
    ).add_to(map)
map

함수를 이용하여 마커와 팝업을 찍어내 보았습니다. 서울시의 종합병원은 중앙쪽에 몰려있고 외곽쪽으로 갈수록 적어진다는 것을 알 수 있었습니다.